In [2]:
import numpy as np
import os
from skimage.io import imread, imsave
from skimage.draw import rectangle
from path_to_folders import images_path, labels_path, masks_path

### Create mask function

In [46]:
def create_mask(image_shape, labels):
    #Create mask of zero
    mask = np.zeros(image_shape[:2], dtype = np.uint8)

    height, width = image_shape[:2]
    for label in labels:
        _, x_center, y_center, width_norm, heighr_norm = map(float, label.split())

        box_width = int(width_norm * width)
        box_height = int(heighr_norm * height)

        x_center_abs = int(x_center * width)
        y_center_abs = int(y_center * height)

        top_left_x = max(0, int(x_center_abs - box_width / 2))
        top_left_y = max(0, int(y_center_abs - box_height / 2))
        bottom_right_x = min(width, int(x_center_abs + box_width / 2))
        bottom_right_y = min(height, int(y_center_abs + box_height / 2))

        rr, cc = rectangle(start=(top_left_y, top_left_x), extent=(bottom_right_y - top_left_y, bottom_right_x - top_left_x), shape=image_shape[:2])
        mask[rr,cc] = 1 # Set the pixels in the rectangle to 1

    return mask

### Function check file in folders and return count files that match

In [47]:
def count_matching_filenames(images_folder, labels_folder):
    # Get a list of filenames without extensions for each folder
    images_file = {os.path.splitext(file)[0] for file in os.listdir(images_folder)}
    label_file = {os.path.splitext(file)[0] for file in os.listdir(labels_folder)}

    # Finding the intersection between two sets
    matching_files = images_file.intersection(label_file)
    
    return len(matching_files)


In [48]:
def process_images(images_folder, labels_folder, masks_folder, image_type, label_type): 
    
    for i in range(count_matching_filenames(images_folder, labels_folder)):
        img_path = os.path.join(images_folder, f'{i}.{image_type}')
        lbl_path = os.path.join(labels_folder, f'{i}.{label_type}')

        if os.path.exists(img_path) and os.path.exists(lbl_path):
            image = imread(img_path)
            with open(lbl_path, 'r') as file:
                labels = file.readlines()
            
            mask = create_mask(image.shape, labels)
            output_folder = os.path.join(masks_folder, f'{i}_mask.png')
            imsave(output_folder, mask)

    # Create the source folder if it does not exist
    if not os.path.exists(masks_folder):
        os.makedirs(output_folder)

### Process images function

In [ ]:
# Image processing
process_images(images_path, labels_path, masks_path, 'jpg', 'txt')